# Install Feast on Kubernetes with the Feast Operator
## Objective

Provide a reference implementation of a runbook to deploy a Feast environment on a Kubernetes cluster using the [Feast Operator](../../infra/feast-operator/).

## Prerequisites
* Kubernetes Cluster
* [kubectl](https://kubernetes.io/docs/tasks/tools/#kubectl) Kubernetes CLI tool.

## Install Prerequisites

The following commands install and configure all the prerequisites on a MacOS environment. You can find the
equivalent instructions on the offical documentation pages:
* Install the `kubectl` cli.
* Install Kubernetes and Container runtime (e.g. [Colima](https://github.com/abiosoft/colima)).
  * Alternatively, authenticate to an existing Kubernetes or OpenShift cluster.
  
```bash
brew install colima kubectl
colima start -r containerd -k -m 3 -d 100 -c 2 --cpu-type max -a x86_64
colima list
```

In [1]:
!kubectl create ns feast
!kubectl config set-context --current --namespace feast

namespace/feast created
Context "colima" modified.


Validate the cluster setup:

In [2]:
!kubectl get ns feast

NAME    STATUS   AGE
feast   Active   3s


## Deployment Architecture
The primary objective of this runbook is to guide the deployment of Feast services on a Kubernetes cluster, using the `postgres` template to set up a basic feature store.

In this notebook, we will deploy a distributed topology of Feast services, which includes:

* `Registry Server`: Handles metadata storage for feature definitions.
* `Online Store Server`: Uses the `Registry Server` to query metadata and is responsible for low-latency serving of features.
* `Offline Store Server`: Uses the `Registry Server` to query metadata and provides access to batch data for historical feature retrieval.

Each service is backed by a `PostgreSQL` database, which is also deployed within the same cluster.

## Setup Postgresql and Redis
Apply the included [postgres](postgres.yaml) & [redis](redis.yaml) deployments to run simple databases.

In [3]:
!kubectl apply -f postgres.yaml -f redis.yaml
!kubectl wait --for=condition=available --timeout=5m deployment/redis
!kubectl wait --for=condition=available --timeout=5m deployment/postgres

secret/postgres-secret created
deployment.apps/postgres created
service/postgres created
deployment.apps/redis created
service/redis created
deployment.apps/redis condition met
deployment.apps/postgres condition met


In [4]:
!kubectl get all

NAME                           READY   STATUS    RESTARTS   AGE
pod/postgres-ff8d4cf48-rvp86   1/1     Running   0          71s
pod/redis-b4756b75d-m5l96      1/1     Running   0          70s

NAME               TYPE        CLUSTER-IP      EXTERNAL-IP   PORT(S)    AGE
service/postgres   ClusterIP   10.43.193.169   <none>        5432/TCP   71s
service/redis      ClusterIP   10.43.64.8      <none>        6379/TCP   69s

NAME                       READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/postgres   1/1     1            1           71s
deployment.apps/redis      1/1     1            1           70s

NAME                                 DESIRED   CURRENT   READY   AGE
replicaset.apps/postgres-ff8d4cf48   1         1         1       71s
replicaset.apps/redis-b4756b75d      1         1         1       70s


## Install the Feast Operator

In [6]:
## Use this install command from a release branch (e.g. 'v0.43-branch')
!kubectl apply -f ../../infra/feast-operator/dist/install.yaml

## OR, for the latest code/builds, use one the following commands from the 'master' branch
# !make -C ../../infra/feast-operator install deploy IMG=quay.io/feastdev-ci/feast-operator:develop FS_IMG=quay.io/feastdev-ci/feature-server:develop
# !make -C ../../infra/feast-operator install deploy IMG=quay.io/feastdev-ci/feast-operator:$(git rev-parse HEAD) FS_IMG=quay.io/feastdev-ci/feature-server:$(git rev-parse HEAD)

!kubectl wait --for=condition=available --timeout=5m deployment/feast-operator-controller-manager -n feast-operator-system

namespace/feast-operator-system created
customresourcedefinition.apiextensions.k8s.io/featurestores.feast.dev created
serviceaccount/feast-operator-controller-manager created
role.rbac.authorization.k8s.io/feast-operator-leader-election-role created
clusterrole.rbac.authorization.k8s.io/feast-operator-featurestore-editor-role created
clusterrole.rbac.authorization.k8s.io/feast-operator-featurestore-viewer-role created
clusterrole.rbac.authorization.k8s.io/feast-operator-manager-role created
clusterrole.rbac.authorization.k8s.io/feast-operator-metrics-auth-role created
clusterrole.rbac.authorization.k8s.io/feast-operator-metrics-reader created
rolebinding.rbac.authorization.k8s.io/feast-operator-leader-election-rolebinding created
clusterrolebinding.rbac.authorization.k8s.io/feast-operator-manager-rolebinding created
clusterrolebinding.rbac.authorization.k8s.io/feast-operator-metrics-auth-rolebinding created
service/feast-operator-controller-manager-metrics-service created
deployment.ap

## Install the Feast services via FeatureStore CR
Next, we'll use the running Feast Operator to install the feast services. Apply the included [reference deployment](feast.yaml) to install and configure Feast.

In [1]:
!kubectl apply -f feast.yaml

secret/feast-data-stores created
featurestore.feast.dev/example created


## Validate the running FeatureStore deployment
Validate the deployment status.

In [2]:
!kubectl get all
!kubectl wait --for=condition=available --timeout=8m deployment/feast-example

NAME                                READY   STATUS     RESTARTS   AGE
pod/feast-example-6c6b58474-n62rg   0/1     Init:0/1   0          4s
pod/postgres-ff8d4cf48-rvp86        1/1     Running    0          3m23s
pod/redis-b4756b75d-m5l96           1/1     Running    0          3m22s

NAME                           TYPE        CLUSTER-IP      EXTERNAL-IP   PORT(S)    AGE
service/feast-example-online   ClusterIP   10.43.175.253   <none>        80/TCP     6s
service/postgres               ClusterIP   10.43.193.169   <none>        5432/TCP   3m23s
service/redis                  ClusterIP   10.43.64.8      <none>        6379/TCP   3m21s

NAME                            READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/feast-example   0/1     1            0           5s
deployment.apps/postgres        1/1     1            1           3m23s
deployment.apps/redis           1/1     1            1           3m22s

NAME                                      DESIRED   CURRENT   READY   AGE
replic

Validate that the FeatureStore CR is in a `Ready` state.

In [4]:
!kubectl get feast

NAME      STATUS   AGE
example   Ready    7m40s


Verify that the DB includes the expected tables.

In [1]:
!kubectl exec deploy/postgres -- psql -h localhost -U feast feast -c '\dt'

                List of relations
 Schema |          Name           | Type  | Owner 
--------+-------------------------+-------+-------
 public | data_sources            | table | feast
 public | entities                | table | feast
 public | feast_metadata          | table | feast
 public | feature_services        | table | feast
 public | feature_views           | table | feast
 public | managed_infra           | table | feast
 public | on_demand_feature_views | table | feast
 public | permissions             | table | feast
 public | projects                | table | feast
 public | saved_datasets          | table | feast
 public | stream_feature_views    | table | feast
 public | validation_references   | table | feast
(12 rows)



Finally, let's verify the feast version.

In [2]:
!kubectl exec deployment/feast-example -itc online -- feast version

Feast SDK Version: "0.1.dev1+gcc1fcad.d20250403"
